In [1]:
# Import all necessary libraries
import math # Needed for basic math operations\n",
import pandas as pd # Needed fpr dataframe creation and operations\n",
import numpy as np # Needed for array manipulations\n",
from itertools import islice # Needed for more complex row and coloumn slicing\n",
import matplotlib.pyplot as plt # Needed for quickly ploting results"
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [3]:
# Variable declarations
ohmStakedInit = 10 # Assume we are stating with 20 ohms
desiredUSDTarget = 50000 # This is the desired USD target you want
desiredOHMTarget = 1000 # This is the desired OHM target you want
desiredDailyIncooom = 2500 # This is your desired daily incooom
desiredWeeklyIncooom = 5000 # This is your desired weekly incooom
#================================================================================

rewardYield = round(1/100,5) # current staking reward yield. Curerntly the yield is 0.47% 
rebaseEpochs = 3 # Not really necessary to declare this here, but for reability it works for now
rebaseConst = 1 + rewardYield
currentAPY = 17407/100 # current staking %APY. Need to make this read from a source or user entry

In [4]:
# Pull most recent ohm price from coingecko
from pycoingecko import CoinGeckoAPI # Pulls live data from coingecko
cg = CoinGeckoAPI() # Pulls live price feed from coingecko
ohmPrice = cg.get_price(ids='olympus',vs_currencies='usd') # Get most recent price from coingecko via api
ohmPrice = ohmPrice['olympus']['usd']

In [5]:
 # STEP 1 : create a data frame to hold all required data point. Data required would be Epochs since rebase are distributed every Epoch
ohmGrowth_df = pd.DataFrame(np.arange(3001), columns = ['Epochs']) # In this case let's consider 3000 Epochs which is 1000 days
ohmGrowth_df['Days'] = ohmGrowth_df.Epochs/3 # There are 3 Epochs per day so divide by 3 to get Days
#================================================================================

In [106]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__,external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("Outputs"),
    html.Label('Initial Ohms'),
    dcc.Input(
        id='initialOhms',
        type='number',
        placeholder='1.0',
        value=1,
        min=1, 
        max=10000, 
        step=1
        ),
    dcc.Graph(id='ohmGrowthChart'),
    html.Label('Current Reward Yield'),
    dcc.Input(
        id='rewardYield',
        type='number',
        placeholder='0.3058',
        value=0.3058,
        min=0.01, 
        max=0.4583, 
        step=0.01
        ),
    html.Table([
        html.Tr([html.Td(['Daily ROI: ']), html.Td(id='dailyROI_P')]),
        html.Tr([html.Td(['5 Day ROI: ']), html.Td(id='fivedayROI_P')]),
        html.Tr([html.Td(['7 Day ROI: ']), html.Td(id='sevendayROI_P')]),
        html.Tr([html.Td(['Monthly ROI: ']), html.Td(id='monthlyROI_P')]),
        html.Tr([html.Td(['Annual ROI: ']), html.Td(id='annualROI_P')]),
    ]), 
])

In [107]:
# Ohm growth section: This section calculates and plots the projected ohm growth over time (1000 days)

In [108]:
# Define callback to update the ohmGrowthChart. As user changes the initial staked ohm, the chart updates dynamically 
@app.callback(
    Output('ohmGrowthChart', 'figure'),
    [Input("initialOhms", "value")],
)
#================================================================================

# This function calculates and generates the ohm growth over time using the initial staked ohm as a starting point
def update_figure(initialOhms):
    
# To Calculate the ohm growth over 3000 Epochs or 1000 days, we loop through the exponential ohm growth equation every epoch

    totalOhms = [] # create an empty array that will hold the componded rewards
    
    ohmStakedGrowth = initialOhms # Initial staked ohms used to project growth over time
    for elements in ohmGrowth_df.Epochs: # Initialize the for loop to have loops equal to number of rows or number of epochs
        totalOhms.append(ohmStakedGrowth) # populate the empty array with calclated values each iteration
        ohmStakedGrowth = ohmStakedGrowth*(1+rewardYield) # compound the total amount of ohms 
    ohmGrowth_df['Total_Ohms'] = totalOhms # Clean up and add the new array to the main data frame
    ohmGrowth_df.Days = np.around(ohmGrowth_df.Days,decimals=1) # Python is funny so let's round up our numbers . 1 decimal place for days",
    ohmGrowth_df.Total_Ohms = np.around(ohmGrowth_df.Total_Ohms,decimals=3)  # Python is funny so let's round up our numbers . 3 decimal place for ohms"

#================================================================================  
    
    return px.line(
        ohmGrowth_df, x="Days", y="Total_Ohms",
        render_mode="webgl", title="Accumulated Ohms"
    )

In [109]:
# ROI Section: This section 

In [110]:

# Define callback to update the ohmGrowthChart. As user changes the initial staked ohm, the chart updates dynamically 
@app.callback(
    Output('dailyROI_P', 'children'),
    Output('fivedayROI_P', 'children'),
    Output('sevendayROI_P', 'children'),
    Output('monthlyROI_P', 'children'),
    Output('annualROI_P', 'children'),
    Input("rewardYield", "value")
)
#================================================================================
def update_rewardYield(rewardYield):
    rewardYield = round(rewardYield/100,5)
    # Let's get some ROI Outputs starting with the daily
    dailyROI = (1+rewardYield)**3-1 # Equation to calculate your daily ROI based on reward Yield
    dailyROI_P = round(dailyROI*100,2) # daily ROI in Percentage
    #================================================================================

    # 5 day ROI
    fivedayROI = (1+rewardYield)**(5*3)-1 # Equation to calculate your 5 day ROI based on reward Yield
    fivedayROI_P = round(fivedayROI*100,2) # 5 day ROI in Percentage
    #================================================================================

    # 7 day ROI
    sevendayROI = (1+rewardYield)**(7*3)-1 # Equation to calculate your 7 day ROI based on reward Yield
    sevendayROI_P = round(sevendayROI*100,2) # 7 day ROI in Percentage
    #================================================================================

    # 30 day ROI
    monthlyROI = (1+rewardYield)**(30*3)-1 # Equation to calculate your 30 day ROI based on reward Yield
    monthlyROI_P = round(monthlyROI*100,2) # 30 day ROI in Percentage
    #================================================================================

    # Annual ROI
    annualROI = (1+rewardYield)**(365*3)-1 # Equation to calculate your annual ROI based on reward Yield
    annualROI_P = round(annualROI*100,2) # Equation to calculate your annual ROI based on reward Yield
    #================================================================================

    # Let's create a nice looking table to view the results of our calculations. The table will contain the ROIs and the percentages
    roiData = [['Daily',dailyROI_P],['5 Day',fivedayROI_P],['7 Day',sevendayROI_P],['1 Month',monthlyROI_P],['1 Year',annualROI_P]]
    roiTablulated_df = pd. DataFrame(roiData,columns = ['Cadence','Percentage'])
    #================================================================================
    
    return dailyROI_P,fivedayROI_P,sevendayROI_P,monthlyROI_P,annualROI_P  
  

In [111]:
# Run app and display result inline in the notebook
app.run_server(mode='inline')